# Global Shark Attacks Analysis

First we import all the modules we are going to need during the analysis

In [1]:
import pandas as pd
import numpy as np
import os
import re
import work_files.functions as wf #file with functions created for this particular exercise

To begin with we use pandas to extract the data from our source, csv file, and insert it into a dataframe. We display this dataframe to see how is organized and its shape.

In [2]:
data = pd.read_csv("../your-code/input/GSAF5.csv", encoding="latin")

display(data.head())
display(data.shape)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


(5992, 24)

With a previosuly created function we check which columns contain NaNs and how many. This functions also displays the columns of the dataframe and the dimensions so we can keep track of what we have in the df.

In [3]:
wf.check_nulls(data)

(Country                     43
 Area                       402
 Location                   496
 Activity                   527
 Name                       200
 Sex                        567
 Age                       2681
 Injury                      27
 Fatal (Y/N)                 19
 Time                      3213
 Species                   2934
 Investigator or Source      15
 href formula                 1
 href                         3
 Unnamed: 22               5991
 Unnamed: 23               5990
 dtype: int64,
 Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
        'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
        'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
        'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
        'Unnamed: 23'],
       dtype='object'),
 (5992, 24))

Checking the nulls and the dataframe we can easily see a few columns that are completely irrelevant to our analysis, so we drop them. A couple columns are rename so we will find it easier to refer to them in the future.

In [4]:
data = data.drop(['Time','pdf','href formula', 'href','Unnamed: 22', 'Unnamed: 23'],axis=1)
data = data.rename(columns = {'Sex ':'Sex',"Species ":"Species","Fatal (Y/N)":"Fatal"})

display(data.head())
display(data.shape)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Species,Investigator or Source,Case Number.1,Case Number.2,original order
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c,2016.09.18.c,5993
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b,2016.09.18.b,5992
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a,2016.09.18.a,5991
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,"The Age, 9/18/2016",2016.09.17,2016.09.17,5990
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,2 m shark,"The Age, 9/16/2016",2016.09.16,2016.09.15,5989


(5992, 18)

After a quick inspection I decided to get rid of all data prior to 1950. I consider it to be to irrelevant for this analysis due to its low quality.

In [5]:
data = data[data["Year"] >= 1950]

In [6]:
wf.check_nulls(data)

(Country                     13
 Area                       205
 Location                   260
 Activity                   313
 Name                       104
 Sex                        394
 Age                       1396
 Injury                      15
 Fatal                        8
 Species                   1653
 Investigator or Source      12
 dtype: int64,
 Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
        'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal', 'Species',
        'Investigator or Source', 'Case Number.1', 'Case Number.2',
        'original order'],
       dtype='object'),
 (4271, 18))

## Type

As part of our analysis to improve safety of people around the world we need to differentiate between provoked and unprovoked attacks.  This is the first column we attack and we start doing this by analysing the different values. (This column has no NaNs).

Given the analysis nature I decided to drop the "Invalid" type as it doesn't provide any useful information.

In [7]:
data["Type"].value_counts()

Unprovoked      3193
Provoked         417
Invalid          353
Boat             194
Sea Disaster      95
Boating           19
Name: Type, dtype: int64

In [8]:
data = data[data["Type"] != "Invalid"]

In [9]:
display(data["Type"].value_counts())
data['Type'] = data['Type'].apply(wf.tipo) # function to group Boat and Boating
data["Type"].value_counts()

Unprovoked      3193
Provoked         417
Boat             194
Sea Disaster      95
Boating           19
Name: Type, dtype: int64

Unprovoked    3288
Provoked       417
Boat           213
Name: Type, dtype: int64

## Fatal

We check again the nulls and decide to carry on our cleaning in the column "Fatal", another critical aspect of our analysis. We drop some useless columns which we won't use.

In [10]:
data = data.drop(["Case Number.1","Case Number.2","original order","Investigator or Source"],axis=1)
wf.check_nulls(data)

(Country       10
 Area         184
 Location     231
 Activity     246
 Name          76
 Sex          348
 Age         1236
 Injury        11
 Fatal          4
 Species     1503
 dtype: int64,
 Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
        'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal', 'Species'],
       dtype='object'),
 (3918, 14))

In [11]:
display(data[data["Fatal"].isnull()])
data[data["Fatal"].isnull()].index

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Species
54,2016.06.02.b,02-Jun-16,2016,Unprovoked,AUSTRALIA,New South Wales,Kingscliff,Spearfishing,Waade Madigan and Dr Seyong Kim,M,NaN,"No injury, but sharks repeatedly hit their fin...",NaN,Bronze whaler sharks x 3
3280,1967.09.13,13-Sep-67,1967,Provoked,ITALY,Brindisi Province,Brindisi,Scuba diving,Romeo Guarini,M,21,"Diver shot the shark, then it injured his arm ...",NaN,2 m shark
3901,1958.12.28,28-Dec-58,1958,Boat,TURKEY,Ahirkapi coast,Constantinople,Fishing,Fishing boat. Occupants: Yunus Potur & Ali Durmaz,NaN,NaN,Boat damaged,NaN,White shark
4107,1954.07.27,27-Jul-54,1954,Boat,ITALY,Venice Province,Off Chioggia,Fishing trawler Flavio Gioia,10 crew,M,NaN,No injury to occupants. Shark tore nets & traw...,NaN,5m shark


Int64Index([54, 3280, 3901, 4107], dtype='int64')

As there are only 4 nulls in the fatal columns they can be individually analysed and modified. In this case the injury column gives us enough information to asume this four cases are non-fatal. 

We get rid of all the "UNKNOWN" type inside "Fatal", although in a more serious analysis they could be individualy analyse like we have done for the nulls.

Finally, we fix and error of an extra space in " N". 

In [12]:
data.loc[54,"Fatal"] = "N"
data.loc[3280,"Fatal"] = "N"
data.loc[3901,"Fatal"] = "N"
data.loc[4107,"Fatal"] = "N"

In [13]:
data["Fatal"].value_counts()

N          3253
Y           615
UNKNOWN      43
 N            7
Name: Fatal, dtype: int64

In [14]:
data = data[data["Fatal"] != "UNKNOWN"]
data["Fatal"] = data["Fatal"].apply(wf.espacio)
data["Fatal"].value_counts()

N    3260
Y     615
Name: Fatal, dtype: int64

## Country

Our third step is to clean the "Country" column. Given that we want to know the countries with highest number of attacks we drop all countries with less than 10 attacks as well as the null columns.
We drop "Area" and "Location" columns as well. 

In [15]:
data["Country"].value_counts()

USA                     1621
AUSTRALIA                669
SOUTH AFRICA             406
PAPUA NEW GUINEA         108
BRAZIL                    86
                        ... 
NIGERIA                    1
MEXICO                     1
NETHERLANDS ANTILLES       1
ARUBA                      1
JOHNSTON ISLAND            1
Name: Country, Length: 139, dtype: int64

In [16]:
#def drop_less_10( df, columna = "column"):
keys = data["Country"].value_counts().index
values = data["Country"].value_counts().values
countries = {keys[i]: values[i] for i in range(len(keys))}
drop_countries = []

for i in countries:
    if countries[i] < 10:
        drop_countries.append(i)
        
for c in drop_countries:
    data = data[data["Country"] != c]

#return df No me ha salido hacer esto con una función así que lo hago a la cuenta de la vieja
#drop_less_10(data,columna="Country")

data["Country"].value_counts()

USA                 1621
AUSTRALIA            669
SOUTH AFRICA         406
PAPUA NEW GUINEA     108
BRAZIL                86
BAHAMAS               78
NEW ZEALAND           71
REUNION               53
MEXICO                52
NEW CALEDONIA         45
FIJI                  40
MOZAMBIQUE            35
ITALY                 35
PHILIPPINES           29
EGYPT                 25
JAPAN                 23
HONG KONG             23
FRENCH POLYNESIA      17
SPAIN                 14
PANAMA                13
SOLOMON ISLANDS       13
CUBA                  12
TONGA                 11
VANUATU               10
IRAN                  10
VIETNAM               10
Name: Country, dtype: int64

In [17]:
data = data.dropna(subset=["Country"])
data = data.drop(["Area","Location"],axis=1)

## Activity

For this analysis it is very important to know the activity of the victim in the moment of the attack. First, we check the number of nulls in this column and we proceed to try and clean it.

In [18]:
wf.check_nulls(data)

(Activity     194
 Name          45
 Sex          279
 Age         1030
 Species     1306
 dtype: int64,
 Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Activity', 'Name',
        'Sex', 'Age', 'Injury', 'Fatal', 'Species'],
       dtype='object'),
 (3509, 12))

In [19]:
data["Activity"].value_counts()

Surfing                                    832
Swimming                                   406
Spearfishing                               264
Fishing                                    208
Wading                                     113
                                          ... 
Fishing, stepped on hooked shark's head      1
Catching sardines                            1
canoeing                                     1
Fishing from prawn trawler                   1
Scuba diving (ascending using scooter)       1
Name: Activity, Length: 755, dtype: int64

In [20]:
data['Activity'] = data['Activity'].apply(wf.activities) # function created to group similar activities
# the NaN values are group within "Other"
data["Activity"].value_counts()

Surfing           1130
Other              751
Fishing            717
Swimming           509
Diving             252
Snorkeling          78
Rowing              39
Treading Water      15
Sea Disaster         9
Paddle Sport         5
Air Disaster         4
Name: Activity, dtype: int64

## Sex

Checking the nulls again I decided to tackle the last column needed for my analysis, "Sex".

In [21]:
wf.check_nulls(data)

(Name         45
 Sex         279
 Age        1030
 Species    1306
 dtype: int64,
 Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Activity', 'Name',
        'Sex', 'Age', 'Injury', 'Fatal', 'Species'],
       dtype='object'),
 (3509, 12))

In [22]:
data["Sex"].value_counts()

M      2816
F       411
M         2
lli       1
Name: Sex, dtype: int64

In [23]:
data['Sex'] = data['Sex'].apply(wf.gender) # function created to remove extra spaces: "M "

In [24]:
data["Sex"].value_counts() 

M      2818
F       411
lli       1
Name: Sex, dtype: int64

In [25]:
data[data["Sex"]=="lli"] # We check what happen with this "lli"

,Case Number,Date,Year,Type,Country,Activity,Name,Sex,Age,Injury,Fatal,Species
1400,2004.11.11.b,11-Nov-04,2004,Unprovoked,USA,Surfing,Brian Kang,lli,38,"Lacerations to hand, knee & thigh",N,5.5 m [18'] white shark


In [26]:
data.loc[1400,"Sex"] = "M" #As we can clearly see the victim Brian is a man, we modified it manually

We can see that "Sex" has 279 null values, this will make no difference because even if all of those victims are female the number of male victims is much higher so I decided to keep them anyway.

In [27]:
wf.check_nulls(data) 

(Name         45
 Sex         279
 Age        1030
 Species    1306
 dtype: int64,
 Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Activity', 'Name',
        'Sex', 'Age', 'Injury', 'Fatal', 'Species'],
       dtype='object'),
 (3509, 12))

# Final Part

Finally, we drop all irrelevant columns left in our dataframe and we export our csv with the clean data so we can go ahead and analyse it using Tableau

In [28]:
data = data.drop(["Name","Date","Age","Injury","Species"],axis=1)
wf.check_nulls(data)

(Sex    279
 dtype: int64,
 Index(['Case Number', 'Year', 'Type', 'Country', 'Activity', 'Sex', 'Fatal'], dtype='object'),
 (3509, 7))

In [29]:
display(data.head())
display(data.shape)

,Case Number,Year,Type,Country,Activity,Sex,Fatal
0,2016.09.18.c,2016,Unprovoked,USA,Surfing,M,N
1,2016.09.18.b,2016,Unprovoked,USA,Surfing,M,N
2,2016.09.18.a,2016,Unprovoked,USA,Surfing,M,N
3,2016.09.17,2016,Unprovoked,AUSTRALIA,Surfing,M,N
4,2016.09.15,2016,Unprovoked,AUSTRALIA,Surfing,M,N


(3509, 7)

In [30]:
shark_data = data
shark_data.to_csv(r'../your-code/output/shark_data.csv')

## Tableau Presentation

https://public.tableau.com/profile/ignacio5899#!/vizhome/GlobalSharkAttacks_15673769293240/SharkAttacks?publish=yes